**A few notes before you run the code:**

This code should run completely through as long as you change the filepaths. The only caveat is that the numbers you see here might not match what's in the MSE and train and test score accuracy table. That's because when we created the table, we had set a random state for some but not all of the various checks we did. Basically, whenever we change the random state for the draws of the train and test set, we sometimes get drastically different results of the training scores: either the models completely overfit or the training scores are much lower. We attribute this to the overwhelming noise that's in our dataset which tells us that there's really not a good way to predict casualties using our feature matrix. We can submit screenshots of all of the various training and test set scores we've gotten over the last three days when we didn't set the random state as evidence of this if needed. They vary but all consistently get a negative test set score, regardless of the training set score which tells us the same story about the noise.

In [ ]:
%matplotlib inline
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/My Drive/Econ 484/project'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Econ 484/project


Ignore the warnings (there will be many).

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Import important packages and the dataset.

In [ ]:
import pandas as pd
from sklearn import linear_model
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm
from statsmodels.tools import add_constant
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


shooter = pd.read_csv('Firearms_cleaned_Apr7.csv')
shooter = shooter.drop(['Unnamed: 0'], axis = 1)
casualties = pd.read_csv('numInj_with_index_Apr7.csv', index_col = 'Unnamed: 0')
casualties = casualties.drop(['Case #'], axis = 1)
casualties.rename(columns={'0': 'Casualties'}, inplace=True)

casualties

,Casualties
0,46
1,7
2,12
3,9
4,21
...,...
183,4
184,53
185,7
186,24


In [ ]:
shooter.describe()

,Month,Year,Latitude,Longitude,Insider or Outsider,Workplace Shooting,Multiple Locations,Armed Person on Scene,Family Member Victim,Romantic Partner Victim,...,Day of Week_Saturday_1,Pop Culture Connection_0.0,Pop Culture Connection_1.0,Pop Culture Connection_2.0,Classification,Caliber,Modified,Large Capacity Magazine,Extended Magazine,Illegal Purchase
count,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,...,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000,188.000000
mean,6.622340,2002.303191,37.738308,-96.240575,0.436170,0.308511,0.329787,0.138298,0.090426,0.117021,...,0.117021,0.898936,0.037234,0.063830,1.260638,1.329787,0.191489,0.601064,0.132979,0.159574
std,3.439458,14.744838,6.312035,18.370110,0.497233,0.463112,0.471391,0.346134,0.287556,0.322304,...,0.322304,0.302218,0.189840,0.245102,1.283854,0.772361,0.394524,0.490987,0.340458,0.367189
min,1.000000,1966.000000,21.307760,-157.863492,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1992.000000,33.744608,-115.146079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,7.000000,2005.000000,37.803851,-90.805986,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,1.000000,2.000000,0.000000,1.000000,0.000000,0.000000
75%,10.000000,2015.000000,41.673558,-80.563051,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,3.000000,2.000000,0.000000,1.000000,0.000000,0.000000
max,12.000000,2022.000000,65.018700,-71.067335,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000


There's still a few "quantitative" variables that are actually categorical and need to be made into dummies.

In [ ]:
# Turn the caliber and classification categorical variables into dummy variables
shooter = pd.get_dummies(shooter, prefix = ['Caliber', 'Classification'], columns = ['Caliber', 'Classification'])

# Verify that it worked correctly
for name in shooter.columns:
  print(name)

Month
Year
Latitude
Longitude
Insider or Outsider
Workplace Shooting
Multiple Locations
Armed Person on Scene
Family Member Victim
Romantic Partner Victim
Age
Immigrant
Sexual Orientation
Known to Police or FBI
History of Animal Abuse
History of Sexual Offenses
Gang Affiliation
Terror Group Affiliation
Bully
Bullied
Raised by Single Parent
Parental Divorce / Separation
Parental Death in Childhood
Parental Suicide
Childhood Trauma
Physically Abused
Sexually Abused
Emotionally Abused
Neglected
Mother Violent Treatment
Parental Substance Abuse
Parent Criminal Record
Family Member Incarcerated
Signs of Being in Crisis
Inability to Perform Daily Tasks
Notably Depressed Mood
Unusually Calm or Happy
Rapid Mood Swings
Increased Agitation
Abusive Behavior
Isolation
Losing Touch with Reality
Paranoia
Prior Hospitalization
Prior Counseling
Treatment 6 Months Prior to Shooting
FASD (Fetal Alcohol Spectrum Disorder)
Autism Spectrum
Health Issues
Head Injury / Possible TBI
Motive: Misogyny
Motive: H

**POLYNOMIAL FEATURES**

Also, we need to label the variable names after adding the polynomial features so we run this program.

In [ ]:
from sklearn import preprocessing as pp
from pandas import DataFrame

def PolynomialFeatures_labeled(input_df,power):
    '''Basically this is a cover for the sklearn preprocessing function. 
    The problem with that function is if you give it a labeled dataframe, it ouputs an unlabeled dataframe with potentially
    a whole bunch of unlabeled columns. 
    Inputs:
    input_df = Your labeled pandas dataframe (list of x's not raised to any power) 
    power = what order polynomial you want variables up to. (use the same power as you want entered into pp.PolynomialFeatures(power) directly)
    Ouput:
    Output: This function relies on the powers_ matrix which is one of the preprocessing function's outputs to create logical labels and 
    outputs a labeled pandas dataframe   
    '''
    poly = pp.PolynomialFeatures(power, include_bias = False)
    output_nparray = poly.fit_transform(input_df)
    powers_nparray = poly.powers_

    input_feature_names = list(input_df.columns)
    target_feature_names = ["Constant Term"]
    for feature_distillation in powers_nparray[1:]:
        intermediary_label = ""
        final_label = ""
        for i in range(len(input_feature_names)):
            if feature_distillation[i] == 0:
                continue
            else:
                variable = input_feature_names[i]
                power = feature_distillation[i]
                intermediary_label = "%s^%d" % (variable,power)
                if final_label == "":         #If the final label isn't yet specified
                    final_label = intermediary_label
                else:
                    final_label = final_label + " x " + intermediary_label
        target_feature_names.append(final_label)
    output_df = pd.DataFrame(output_nparray, columns = target_feature_names)
    return output_df

**GUN CLASSIFICATIONS**

In [ ]:
# First, we create three new dataframes with our variables of interest. 
class1 = pd.DataFrame(shooter.loc[:, 'Classification_1.0'])
class2 = pd.DataFrame(shooter.loc[:, 'Classification_2.0'])
class3 = pd.DataFrame(shooter.loc[:, 'Classification_3.0'])

# Next, we drop all of the classification variables from our dataframe - everything will be in reference to handguns which is Classification 0.
newshooter1 = shooter.drop(['Classification_0.0', 'Classification_1.0', 'Classification_2.0', 'Classification_3.0'], axis = 1)

shootertf = newshooter1

# Run it through the PolynomialFeatures labeling function that both transforms it and labels it.
shootertf = PolynomialFeatures_labeled(newshooter1, 2)

Our outputs are as follows:
- df with d variable #1 = class1
- df with d variable #2 = class2
- df with d variable #3 = class3
- transformed df with classifications dropped = shootertf

Classification_3 is an assault weapon, we are probably the most interested in the causal effect of using an assault weapon on # of people killed or injured. That being said, we will be able to get the coefficients on all types of weapons as compared to the base "handguns" which on the mode was used the most often using PDS Lasso and DDML using Ridge.

**ANALYSIS USING CLASSIFICATIONS AS OUR D VARIABLES (EXCLUDING HANDGUNS)**

*PDS Lasso:*

In [ ]:
X1 = shootertf
y = casualties
dshotgun = class1
drifle = class2
dassault = class3

In [ ]:
print(X1.shape)
print(y.shape)
print(dshotgun.shape)
print(drifle.shape)
print(dassault.shape)

(188, 89675)
(188, 1)
(188, 1)
(188, 1)
(188, 1)


In [ ]:
# Standardize the data
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
scaler = StandardScaler()
X1_names = X1.columns
Xtf = pd.DataFrame(scaler.fit_transform(X1))
Xtf.columns = X1_names

In [ ]:
Xtf_train, Xtf_test, y_train, y_test = train_test_split(Xtf, y, random_state = 1000)

In [ ]:
# Run Lasso on a random training set and test set to check the score so that we can compare it to the score we'd get on ridge.
lassocv = LassoCV(cv = 5, max_iter=100000, random_state = 5000).fit(Xtf_train, y_train)

print('Lasso score on training set: {:.4f}'.format(lassocv.score(Xtf_train, y_train)))
print('Lasso score on test set: {:.4f}'.format(lassocv.score(Xtf_test, y_test)))

alpha = lassocv.alpha_
print("Selected alpha value: {:.4f}".format(alpha))

Lasso score on training set: 0.9761
Lasso score on test set: -0.0083
Selected alpha value: 0.4944


In [ ]:
mse_predict_train = lassocv.predict(Xtf_train)
mse_predict_test = lassocv.predict(Xtf_test)

training_mse = mean_squared_error(y_train, mse_predict_train)
test_mse = mean_squared_error(y_test, mse_predict_test)
print("MSE on training set: {:.3f}".format(training_mse))
print("MSE on test set: {:.3f}".format(test_mse))

MSE on training set: 4.288
MSE on test set: 17704.074


*PDS Lasso without Polynomial Features:*

In [ ]:
X2 = newshooter1
y = casualties
dshotgun = class1
drifle = class2
dassault = class3

In [ ]:
# Standardize the data
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
scaler = StandardScaler()
X2_names = X2.columns
Xtf2 = pd.DataFrame(scaler.fit_transform(X2))
Xtf2.columns = X2_names

In [ ]:
Xtf2_train, Xtf2_test, y_train, y_test = train_test_split(Xtf2, y, random_state = 1000)

In [ ]:
# Run Lasso on a random training set and test set to check the score so that we can compare it to the score we'd get on ridge.
lassocv = LassoCV(cv = 5, max_iter=100000).fit(Xtf2_train,y_train)

print('Lasso score on training set: {:.4f}'.format(lassocv.score(Xtf2_train, y_train)))
print('Lasso score on test set: {:.4f}'.format(lassocv.score(Xtf2_test, y_test)))

alpha = lassocv.alpha_
print("Selected alpha value: {:.4f}".format(alpha))

Lasso score on training set: 0.5552
Lasso score on test set: -0.0116
Selected alpha value: 2.1401


In [ ]:
mse_predict_train = lassocv.predict(Xtf2_train)
mse_predict_test = lassocv.predict(Xtf2_test)

training_mse = mean_squared_error(y_train, mse_predict_train)
test_mse = mean_squared_error(y_test, mse_predict_test)
print("MSE on training set: {:.3f}".format(training_mse))
print("MSE on test set: {:.3f}".format(test_mse))

MSE on training set: 79.839
MSE on test set: 17762.978


Double De-biased Machine Learning with RidgeCV?


*Ridge:*

Test if ridge will work any better than lasso: it's unlikely that it will.

In [ ]:
Xtf2_train, Xtf2_test, y_train, y_test = train_test_split(Xtf2, y, random_state = 1000)

In [ ]:
ridgecv = RidgeCV(cv = 5).fit(Xtf2_train,y_train)
print('Ridge score on training set: {:.4f}'.format(ridgecv.score(Xtf2_train, y_train)))
print('Ridge score on test set: {:.4f}'.format(ridgecv.score(Xtf2_test,y_test)))

Ridge score on training set: 0.9985
Ridge score on test set: -0.0011


In [ ]:
mse_predict_train = ridgecv.predict(Xtf2_train)
mse_predict_test = ridgecv.predict(Xtf2_test)

training_mse = mean_squared_error(y_train, mse_predict_train)
test_mse = mean_squared_error(y_test, mse_predict_test)
print("MSE on training set: {:.3f}".format(training_mse))
print("MSE on test set: {:.3f}".format(test_mse))

MSE on training set: 0.276
MSE on test set: 17578.657


*Ridge with Polynomial Features:*

In [ ]:
ridgecv = RidgeCV(cv = 5).fit(Xtf_train,y_train)
print('Ridge score on training set: {:.4f}'.format(ridgecv.score(Xtf_train, y_train)))
print('Ridge score on test set: {:.4f}'.format(ridgecv.score(Xtf_test,y_test)))

Ridge score on training set: 0.9985
Ridge score on test set: -0.0011


In [ ]:
mse_predict_train = ridgecv.predict(Xtf_train)
mse_predict_test = ridgecv.predict(Xtf_test)

training_mse = mean_squared_error(y_train, mse_predict_train)
test_mse = mean_squared_error(y_test, mse_predict_test)
print("MSE on training set: {:.3f}".format(training_mse))
print("MSE on test set: {:.3f}".format(test_mse))

MSE on training set: 0.276
MSE on test set: 17578.657


Double De-biased Machine Learning with Random Forest:

*Gradient Boosted Regressor*

In [ ]:
gbt = GradientBoostingRegressor(n_estimators = 100, random_state = 0, max_depth = 2)
gbt.fit(Xtf2_train, y_train) 

print("Accuracy on training set: {:.4f}".format(gbt.score(Xtf2_train, y_train)))
print("Accuracy on test set: {:.4f}".format(gbt.score(Xtf2_test, y_test)))

Accuracy on training set: 0.9388
Accuracy on test set: -0.0072


In [ ]:
mse_predict_train = gbt.predict(Xtf2_train)
mse_predict_test = gbt.predict(Xtf2_test)

training_mse = mean_squared_error(y_train, mse_predict_train)
test_mse = mean_squared_error(y_test, mse_predict_test)
print("MSE on training set: {:.3f}".format(training_mse))
print("MSE on test set: {:.3f}".format(test_mse))

MSE on training set: 10.981
MSE on test set: 17684.551


*Random Forest with Polynomial Features*

Now test to see if random forest can fix our problem. But first, cross-validate!

In [ ]:
from sklearn.model_selection import GridSearchCV

tree_complexity_grid = {'max_depth': np.arange(1,10,1),'min_samples_split': np.arange(1,3,1), 'min_samples_leaf': np.arange(1,5,1)}
grid_search = GridSearchCV(RandomForestRegressor(), tree_complexity_grid, cv=5, return_train_score=True)
best_model=grid_search.fit(Xtf_train,y_train)
print("Best minimum samples to split: ",best_model.best_estimator_.get_params()['min_samples_split'])
print("Best max depth: ",best_model.best_estimator_.get_params()['max_depth'])
print("Best minimum leaves to split: ",best_model.best_estimator_.get_params()['min_samples_leaf'])

Best minimum samples to split:  2
Best max depth:  8
Best minimum leaves to split:  1


In [ ]:
forest = RandomForestRegressor(n_estimators = 100, random_state = 1000, max_depth = 8, min_samples_split = 2, min_samples_leaf = 1) 
forest.fit(Xtf_train, y_train)

print("Accuracy on training set: {:.4f}".format(forest.score(Xtf_train, y_train))) 
print("Accuracy on test set: {:.4f}".format(forest.score(Xtf_test, y_test)))

Accuracy on training set: 0.8860
Accuracy on test set: -0.0049


In [ ]:
mse_predict_train = forest.predict(Xtf_train)
mse_predict_test = forest.predict(Xtf_test)

training_mse = mean_squared_error(y_train, mse_predict_train)
test_mse = mean_squared_error(y_test, mse_predict_test)
print("MSE on training set: {:.3f}".format(training_mse))
print("MSE on test set: {:.3f}".format(test_mse))

MSE on training set: 20.463
MSE on test set: 17644.195


*Random Forest without Polynomial Features:*

In [ ]:
forest = RandomForestRegressor(n_estimators = 100, random_state = 1000, max_depth = 8, min_samples_split = 2, min_samples_leaf = 1) 
forest.fit(Xtf2_train, y_train)

print("Accuracy on training set: {:.4f}".format(forest.score(Xtf2_train, y_train))) 
print("Accuracy on test set: {:.4f}".format(forest.score(Xtf2_test, y_test)))


Accuracy on training set: 0.8849
Accuracy on test set: -0.0048


In [ ]:
mse_predict_train = forest.predict(Xtf2_train)
mse_predict_test = forest.predict(Xtf2_test)

training_mse = mean_squared_error(y_train, mse_predict_train)
test_mse = mean_squared_error(y_test, mse_predict_test)
print("MSE on training set: {:.3f}".format(training_mse))
print("MSE on test set: {:.3f}".format(test_mse))

MSE on training set: 20.666
MSE on test set: 17642.266


Nope. Unexpected, and it was worth a shot but it's even worse! (Fyi, we didn't do cross-validation on any of this because it's unlikely to fix such a bad test set score). If it worked, we would have done sample-splitting (as seen below) but it didn't so we don't.

In [ ]:
# Import random forest
from sklearn.ensemble import RandomForestRegressor
# Instantiate random forest objects
rfy=RandomForestRegressor(n_estimators=100, max_depth = 4)
rfd = rfy

# Create our sample splitting "object"
kf = KFold(n_splits=5,shuffle=True,random_state=42)

# Apply the splits to our feature matrix
kf.get_n_splits(X2)

# Initialize columns for residuals
yresidrf = y*0
dshotgunrf = dshotgun*0
driflerf = drifle*0
dassaultrf = dassault*0

In [ ]:
# Now loop through each fold
for train_index, test_index in kf.split(X2):
  X_train, X_test = X2.iloc[train_index,:], X2.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  dshotgun_train, dshotgun_test = dshotgun.iloc[train_index,:], dshotgun.iloc[test_index,:]
  drifle_train, drifle_test = drifle.iloc[train_index,:], drifle.iloc[test_index,:]
  dassault_train, dassault_test = dassault.iloc[train_index,:], dassault.iloc[test_index,:]
  
  # Do DML thing
  # Random Forest y on training folds:
  rfy.fit(X_train, y_train)
  # But get the residuals in the test set
  yresidrf.iloc[test_index] = y_test - rfy.predict(X_test).reshape(-1,1)
  
  # Random Forest classification 1 (shotguns) on training folds
  rfd.fit(X_train, dshotgun_train)
  # But get residuals on the test set
  dshotgunrf.iloc[test_index,:]=dshotgun_test - rfd.predict(X_test).reshape(-1,1)

  # Random Forest classification 2 (rifles) on training folds
  rfd.fit(X_train, drifle_train)
  # But get residuals on the test set
  driflerf.iloc[test_index,:] = drifle_test - rfd.predict(X_test).reshape(-1,1)

  # Random Forest classification 3 (assault weapons) on training folds
  rfd.fit(X_train, dassault_train)
  # But get residuals on the test set
  dassaultrf.iloc[test_index,:] = dassault_test - rfd.predict(X_test).reshape(-1,1)

In [ ]:
# Regress residuals
rfvars = pd.concat([dshotgunrf, driflerf, dassaultrf], axis = 1)
rfvars = sm.add_constant(rfvars)
lmrf = sm.OLS(yresidrf, rfvars)
rfols = lmrf.fit()
print(rfols.summary())

                            OLS Regression Results                            
Dep. Variable:             Casualties   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.176
Date:                Wed, 19 Apr 2023   Prob (F-statistic):              0.320
Time:                        16:03:45   Log-Likelihood:                -1059.5
No. Observations:                 188   AIC:                             2127.
Df Residuals:                     184   BIC:                             2140.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  3.6199      5

### **So, we do OLS...**

In [ ]:
classes = pd.concat([class1, class2, class3], axis = 1)
classes = sm.add_constant(classes)
lmrf = sm.OLS(casualties, classes)
rfols = lmrf.fit()
print(rfols.summary())

                            OLS Regression Results                            
Dep. Variable:             Casualties   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     2.095
Date:                Wed, 19 Apr 2023   Prob (F-statistic):              0.102
Time:                        16:03:45   Log-Likelihood:                -1056.5
No. Observations:                 188   AIC:                             2121.
Df Residuals:                     184   BIC:                             2134.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 10.4588      7